In [2]:
import json
with open('jan10-v2-hp_formatted_answers_499.jsonl', 'r') as f:
    data = [json.loads(line) for line in f.readlines()]
data[0]

{'raw_question': "What is the name of Ron Weasley's rat?",
 'few_shot_question': "\nI want you to answer the following question about the Harry Potter series. I will provide you with the question, and you will respond with the answer to the question. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\nWhat is the name of Harry Potter's owl?\nAnswer:\nHedwig\n\n\nQuestion:\nWho teaches Potions at Hogwarts when Harry first arrives?\nAnswer:\nSeverus Snape\n\n\nQuestion:\nWhat position does Harry play on his Quidditch team?\nAnswer:\nSeeker\n\n\nQuestion:\nWhat is the name of Ron Weasley's rat?\nAnswer:\n",
 'zero_shot_question': "\nI want you to answer the following question about Harry Potter and respond with an answer. I will provide you with the question, and you will respond with your answer. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\nWhat is the name of Ron Weasley's rat?\n\nAnswer

In [3]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client(
    organization='org-X6T6Ar6geRtOrQgQTQS3OUpw',
)

SYSTEM_MESSAGE = """
I want you to make a judgement on the correctness of a given answer by a contestant. The question pertains to the Harry Potter universe. The contestant has been asked a question and has responded with their answer. Using the provided "Perfect Answer" and your own knowledge, you will decide whether the contestant's response is correct. Your output will be either `Y` or `N` for yes or no. You must not output anything else.
"""
USER_MESSAGE = """
Question: {question}
Contestant's Answer: {answer}
Perfect Answer: {perfect_answer}
"""

def get_model_grade(question, response, perfect_answer, model='gpt-3.5-turbo', max_tokens=1):

    system_message = SYSTEM_MESSAGE
    user_message = USER_MESSAGE.format(question=question, answer=response, perfect_answer=perfect_answer)

    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0,
    seed=42,
    )
    return response.choices[0].message.content

get_model_grade(
    question="What is the name of the wizarding school in Scotland?",
    response="Poop",
    perfect_answer="Hogwarts School of Witchcraft and Wizardry",
)

'N'

In [7]:
with open('jan10-v2-hp_formatted_answers_499.jsonl', 'r') as f:
    data = [json.loads(line) for line in f.readlines()]
for i, datapoint in enumerate(data):
    new_format = {
        'questions': {
            'raw_question': datapoint['raw_question'],
            'perfect_answer': datapoint['perfect_answer'],
            'few_shot_question': datapoint['few_shot_question'],
            'zero_shot_question': datapoint['zero_shot_question'],
            'unrelated_few_shot_question': datapoint['unrelated_few_shot_question'],
        },
        'results':{
            'llama-7b': {
                'few_shot':{
                    'response': datapoint['llama-7b']['few_shot']['response'],
                    'model_grade': datapoint['llama-7b']['few_shot']['model_grade'],
                },
                'zero_shot':{
                    'response': datapoint['llama-7b']['zero_shot']['response'],
                    'model_grade': datapoint['llama-7b']['zero_shot']['model_grade'],
                },
                'unrelated_few_shot':{
                    'response': datapoint['llama-7b']['unrelated_few_shot']['response'],
                    'model_grade': datapoint['llama-7b']['unrelated_few_shot']['model_grade'],
                },
            },
            'hp-7b': {
                'few_shot':{
                    'response': datapoint['hp-7b']['few_shot']['response'],
                    'model_grade': datapoint['hp-7b']['few_shot']['model_grade'],
                },
                'zero_shot':{
                    'response': datapoint['hp-7b']['zero_shot']['response'],
                    'model_grade': datapoint['hp-7b']['zero_shot']['model_grade'],
                },
                'unrelated_few_shot':{
                    'response': datapoint['hp-7b']['unrelated_few_shot']['response'],
                    'model_grade': datapoint['hp-7b']['unrelated_few_shot']['model_grade'],
                },
            },
        }
    }

    data[i] = new_format
data[0]

{'questions': {'raw_question': "What is the name of Ron Weasley's rat?",
  'perfect_answer': 'Scabbers',
  'few_shot_question': "\nI want you to answer the following question about the Harry Potter series. I will provide you with the question, and you will respond with the answer to the question. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\nWhat is the name of Harry Potter's owl?\nAnswer:\nHedwig\n\n\nQuestion:\nWho teaches Potions at Hogwarts when Harry first arrives?\nAnswer:\nSeverus Snape\n\n\nQuestion:\nWhat position does Harry play on his Quidditch team?\nAnswer:\nSeeker\n\n\nQuestion:\nWhat is the name of Ron Weasley's rat?\nAnswer:\n",
  'zero_shot_question': "\nI want you to answer the following question about Harry Potter and respond with an answer. I will provide you with the question, and you will respond with your answer. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\n

In [12]:
from tqdm import tqdm

def save_file(path, data):
    with open(path, 'w') as f:
        for line in data:
            json.dump(line, f)
            f.write('\n')

for i, datapoint in tqdm(enumerate(data)):
    print(f'Iteration {i}/499')
    raw_question = datapoint['questions']['raw_question']
    for model in datapoint['results'].keys():
        for question_type in datapoint['results'][model].keys():
            response = datapoint['results'][model][question_type]['response']
            perfect_answer = datapoint['questions']['perfect_answer']
            datapoint['results'][model][question_type]['model_grade'] = get_model_grade(
                question=raw_question,
                response=response,
                perfect_answer=perfect_answer,
            )
    data[i] = datapoint
    save_file('jan10-v3-hp_formatted_answers_499.jsonl', data)

0it [00:00, ?it/s]

Iteration 0/499


1it [00:02,  2.30s/it]

Iteration 1/499


2it [00:04,  2.13s/it]

...

497it [15:03,  1.83s/it]

Iteration 497/499


498it [15:04,  1.79s/it]

Iteration 498/499


499it [15:06,  1.82s/it]


In [10]:
data[0]

{'questions': {'raw_question': "What is the name of Ron Weasley's rat?",
  'perfect_answer': 'Scabbers',
  'few_shot_question': "\nI want you to answer the following question about the Harry Potter series. I will provide you with the question, and you will respond with the answer to the question. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\nWhat is the name of Harry Potter's owl?\nAnswer:\nHedwig\n\n\nQuestion:\nWho teaches Potions at Hogwarts when Harry first arrives?\nAnswer:\nSeverus Snape\n\n\nQuestion:\nWhat position does Harry play on his Quidditch team?\nAnswer:\nSeeker\n\n\nQuestion:\nWhat is the name of Ron Weasley's rat?\nAnswer:\n",
  'zero_shot_question': "\nI want you to answer the following question about Harry Potter and respond with an answer. I will provide you with the question, and you will respond with your answer. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\n

In [2]:
import json
with open('harry_potter_trivia_502.jsonl', 'r') as f:
    trivia_data = [json.loads(line) for line in f.readlines()]
trivia_data[0]

{'question': "What is the name of Harry Potter's owl?",
 'true_answer': 'Hedwig',
 'false_answer': 'Fluffy'}

In [11]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client(
    organization='org-X6T6Ar6geRtOrQgQTQS3OUpw',
)
USER_MESSAGE = """
Question: {question}
True Answer: {true_answer}
False Answer: {false_answer}
"""

SYSTEM_MESSAGE = f"""
I want you to change the false answers to be sensible but not at all related to the Harry Potter universe. Only return the false answer. You must not output anything else.

An example prompt and response is below:
{USER_MESSAGE.format(question='What is the name of the wizarding school in Scotlansd?', true_answer='Hogwarts School of Witchcraft and Wizardry', false_answer="Muggle High School")}

St Mary's College
"""


def get_model_grade(question, true_answer, false_answer, model='gpt-4', max_tokens=1):

    system_message = SYSTEM_MESSAGE
    user_message = USER_MESSAGE.format(question=question, true_answer=true_answer, false_answer=false_answer)

    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0,
    seed=42,
    )
    return response.choices[0].message.content

for i in range(5):
    false_answer = get_model_grade(
        question=trivia_data[i]['question'],
        true_answer=trivia_data[i]['true_answer'],
        false_answer=trivia_data[i]['false_answer'],
    )
    print(false_answer)

Feathered Fred
Professor Charles Xavier
Striker
Fluffy Paws
John Smith


In [14]:
from tqdm import tqdm
def save_file(path, data):
    with open(path, 'w') as f:
        for line in data:
            json.dump(line, f)
            f.write('\n')
for i, datapoint in tqdm(enumerate(trivia_data)):
    print(f'Iteration {i+1}/502')
    false_answer = get_model_grade(
        question=datapoint['question'],
        true_answer=datapoint['true_answer'],
        false_answer=datapoint['false_answer'],
    )
    trivia_data[i]['false_answer'] = false_answer
    save_file('harry_potter_trivia_502_v2.jsonl', trivia_data)

0it [00:00, ?it/s]

Iteration 1/502


1it [00:00,  1.64it/s]

Iteration 2/502


2it [00:01,  1.01s/it]

...

500it [05:09,  1.79it/s]

Iteration 501/502


501it [05:10,  1.93it/s]

Iteration 502/502


502it [05:11,  1.61it/s]
502it [05:11,  1.61it/s]
